In [1]:
import numpy as np
import mayfly as mf
import h5py
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os 
import sys
import json
import scipy.signal
import scipy.stats
import scipy.interpolate
import pickle as pkl
import scipy.optimize

PATH = '/storage/home/adz6/group/project/'
RESULTPATH = os.path.join(PATH, 'results/mayfly')
PLOTPATH = os.path.join(PATH, 'plots/mayfly')
DATAPATH = os.path.join(PATH, 'datasets/data')


def linear_fit(x, a, b):
    
    return a + b * x



# load data

In [2]:
os.listdir(os.path.join(RESULTPATH, 'scores'))

['210610_epa_grid_self_scores.pkl',
 '210810_mf_scores_84_100.npy',
 '210810_mf_scores_84_100_2.npy',
 '211005_mf_84_100_template_ideal_scores_bottom_tri.npy',
 '211007_mf_84_100_test_scores.npy',
 '211104_84_25_2cm_template_on_template.npy',
 '211106_84_25_2cm_low_energy_on_template.npy',
 '211106_84_25_2cm_random_on_template.npy',
 '211108_84_25_2cm_template_on_template_1slice.npy',
 '211112_84_25_self_scores.npy',
 '211112_84_2cm_low_energy_self_scores.npy',
 '220124_sens_est_dense_grid_84.5_0cm_template_scores_nslice2',
 '220124_sens_est_dense_grid_84.5_1cm_template_scores_nslice2',
 '220124_sens_est_dense_grid_84.5_4cm_template_scores_nslice2',
 '220124_sens_est_dense_grid_86.0_1cm_template_only_scores_nslice2_nsample1024.npy',
 '220124_sens_est_dense_grid_86.0_1cm_template_only_scores_nslice2_nsample2048.npy',
 '220124_sens_est_dense_grid_86.0_1cm_template_only_scores_nslice2_nsample4096.npy',
 '220124_sens_est_dense_grid_86.0_1cm_template_only_scores_nslice2_nsample8192.npy',
 '

In [3]:
os.listdir(os.path.join(DATAPATH))

['211105_84_25_2cm_random.h5',
 '211116_grad_b_est.h5',
 '220405_dl_grid_87deg_10sec_range_10eV_range_20mm_10mm_range_train.h5',
 '220414_dl_grid_84to90_1to45mm.h5',
 '220416_dl_grid_87.5deg_10sec_range_10eV_range_20mm_10mm_range_train.h5',
 '220523_dl_grid_84to90_1to45mm_test.h5',
 '220523_dl_grid_84to90_1to45mm_validation.h5',
 '220604_dl_grid_85to88_5mm.h5',
 '220604_dl_grid_85to88_5mm_new.h5',
 '220609_dl_test_data_85to88deg_18575to18580ev_5mm_random.h5',
 '220619_dl_test_data_85to88deg_18575to18580ev_5mm_random.h5',
 '220819_modify_file.py',
 '220820_combine_files.py',
 '220826_dl_test_data_88to90deg_18575to18580ev_5mm_random.h5',
 '220826_dl_train_data_88to90deg_18575to18580ev_5mm_random.h5',
 '220901_dl_test_data_85to90deg_5mm.h5',
 '220901_dl_train_data_85to90deg_5mm.h5',
 '220913_dl_grid_data_88to90',
 '220913_dl_grid_data_88to90deg_18575to18580ev_5mm.h5',
 '220914_dl_grid_data_88to90',
 '220922_dl_grid_data_85to90deg_18575to18580ev_5mm.h5',
 '221024_test_signal.npy',
 '221024

In [4]:
template_data = mf.data.MFDataset(os.path.join(DATAPATH, '211106_84_2cm_low_energy.h5'))

#beamforming_result = np.load(os.path.join(RESULTPATH,'211005_mf_84_100_template_ideal_scores_bottom_tri.npy'))

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/storage/home/adz6/group/project/datasets/data/211106_84_2cm_low_energy.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
template_metadata = pd.DataFrame(template_data.metadata)

In [ ]:
scores.argmax(axis=-1).shape

In [5]:
scores = np.load(os.path.join(RESULTPATH, 'scores', '211106_84_25_2cm_low_energy_on_template.npy'))
ideal_scores = np.load(os.path.join(RESULTPATH, 'scores', '211112_84_25_self_scores.npy'))


match = scores.max(axis=-1) / ideal_scores[scores.argmax(axis=-1)]

In [9]:
scores.shape

(12186, 12221)

# pick signal, sort scores with metadata, normalize

In [6]:


energy_array = template_metadata['energy'].array
angle_array = template_metadata['theta_min'].array

unique_energy = np.sort(np.unique(template_metadata['energy'].array))
unique_angle = np.sort(np.unique(template_metadata['theta_min'].array))

energy_grid, angle_grid = np.meshgrid(unique_energy, unique_angle)

sorted_match = np.zeros(energy_grid.shape).flatten()

for i, pair in enumerate(zip(energy_grid.flatten(), angle_grid.flatten())):
    
    try:
        index = template_metadata[(template_metadata['energy'] == pair[0]) & (template_metadata['theta_min'] == pair[1])].index[0]
        sorted_match[i] = match[index]
    except:
        sorted_match[i] = 0.0
        
sorted_match = sorted_match.reshape(energy_grid.shape)

NameError: name 'template_metadata' is not defined

# plot

In [ ]:
sns.set_theme(style='ticks', context='talk')
cmap = sns.color_palette('mako', as_cmap=True)
fig = plt.figure(figsize=(13,8))

from matplotlib.colors import LogNorm

ax = fig.add_subplot(1,1,1)
img = ax.imshow(sorted_match, aspect='auto', interpolation='none', cmap=cmap, 
                norm=None, extent=(17574 - 18575, 18574 - 18575, 90, 84))
cbar = fig.colorbar(img)

ax.set_xlabel('Energy Below Window(eV)')
ax.set_ylabel('Pitch Angle (deg)')
name = f'211112_offaxis2cm_low_energy_3timeslice_match_ratio_25eV_window.png'
#ax.plot(signal_energy, signal_angle, 'r.')

plt.tight_layout()

#plt.savefig(os.path.join(PLOTPATH, name))

# plot qualitative

In [ ]:
sns.set_theme(style='ticks', context='talk')
cmap = sns.color_palette('mako', as_cmap=True)
fig = plt.figure(figsize=(13,8))

from matplotlib.colors import LogNorm

ax = fig.add_subplot(1,1,1)
img = ax.imshow(sorted_match, aspect='auto', interpolation='none', cmap='Paired', 
                norm=None, extent=(17574 - 18575, 18574 - 18575, 90, 84))
cbar = fig.colorbar(img)

ax.set_xlabel('Energy Below Window(eV)')
ax.set_ylabel('Pitch Angle (deg)')
name = f'211112_offaxis2cm_low_energy_3timeslice_match_ratio_25eV_window_qualitative.png'
#ax.plot(signal_energy, signal_angle, 'r.')

plt.tight_layout()

#plt.savefig(os.path.join(PLOTPATH, name))

In [ ]:
x

sorted_match[np.argwhere(sorted_match>0.8)[]].shape